### Importing required libraries

In [1]:
from src.utils import heart_slice, plot_function, plot_vector_field
from src.models.dynamics_models import BidomainModel
from src.models.cell_models import *
import ufl

### Defining a domain, cell model and the whole model and visualizing initial data

In [2]:
domain = heart_slice()
cell_model = Noble(domain=domain)
x = ufl.SpatialCoordinate(domain)


class Model(BidomainModel):
    def initial_V_m(self):
        return lambda x: (x[0]-0)** 2 + (x[1]-2.7) ** 2 < 0.4**2, -60, -84

    def conductivity(
        self,
        longitudinal_fibres=[
            x[1] / (x[0] ** 2 + x[1] ** 2),
            -x[0] / (x[0] ** 2 + x[1] ** 2),
        ],
        transversal_fibres=[0, 0],
    ):
        return super().conductivity(longitudinal_fibres, transversal_fibres)

    def ischemia(self):
        return (
            lambda x: (x[0] - 2) ** 2 / 10 + (x[1] - 1.2) ** 2 < 0.4**2,
            self.M_i / 10,
            self.M_e / 10,
        )


model = Model(domain, cell_model)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Nurb)
Info    : [ 40%] Meshing curve 2 (Nurb)
Info    : [ 70%] Meshing curve 3 (Nurb)
Info    : Done meshing 1D (Wall 0.0254163s, CPU 0.022947s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.0720873s, CPU 0.06807s)
Info    : Meshing 3D...
Info    : Done meshing 3D (Wall 2.0801e-05s, CPU 1.1e-05s)
Info    : 3946 nodes 7867 elements


In [3]:
model.plot_initial_V_m(show_mesh=False)

Widget(value='<iframe src="http://localhost:42539/index.html?ui=P_0x7f6e3d33b610_0&reconnect=auto" class="pyvi…

In [4]:
model.plot_ischemia(show_mesh=False)

Widget(value='<iframe src="http://localhost:42539/index.html?ui=P_0x7f6e3a4dfa10_1&reconnect=auto" class="pyvi…

In [5]:
x_c, y_c = 0, 0.8
plot_vector_field(
    domain,
    lambda x: [
        (x[1] - y_c) / ((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        -(x[0] - x_c) / ((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        0,
    ],
    tolerance=0.03,
    factor=1,
)

Widget(value='<iframe src="http://localhost:42539/index.html?ui=P_0x7f6e3a4ee510_2&reconnect=auto" class="pyvi…

### Solving equations with given parameters

In [6]:
model.solve(
    T=1000,
    steps=3000,
    save_to="demo.mp4",
    # signal_point=[0.2, 0.2, 0.0],
)

Solving problem: 100%|██████████| 3000/3000 [15:12<00:00,  3.29it/s]


### Plotting a final plot of transmembrane potential

In [7]:
plot_function(model.V_m_n, "V_m")

Widget(value='<iframe src="http://localhost:42539/index.html?ui=P_0x7f6d978efc90_3&reconnect=auto" class="pyvi…

### Plotting a transmembrane potential in a given signal point

In [8]:
model.plot_signal()

ValueError: Signal point must be specified when solving the model.

ERROR:wslink.protocol:Exception raised
ERROR:wslink.protocol:KeyError('d0e16d18bda784969d2f1038789c6367_15259d')
ERROR:wslink.protocol:Traceback (most recent call last):
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/wslink/protocol.py", line 332, in onMessage
    results = func(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/trame_vtk/modules/vtk/protocols/local_rendering.py", line 33, in get_array
    self.context.get_cached_data_array(data_hash, binary)
  File "/home/drvinko/miniconda3/envs/diplomski/lib/python3.11/site-packages/trame_vtk/modules/vtk/serializers/synchronization_context.py", line 30, in get_cached_data_array
    cache_obj = self.data_array_cache[p_md5]
                ~~~~~~~~~~~~~~~~~~~~~^^^^^^^
KeyError: 'd0e16d18bda784969d2f1038789c6367_15259d'

ERROR:wslink.protocol:Exception raised
ERROR:wslink.protocol:KeyError('855c0cd3d6c53022f19f13f3590e3124_15259d

### Plotting a transmembrane potential solution of a cell model

In [ ]:
cell_model.visualize(350, -60)